In [5]:
# Lets Install patchify package to make smaller imgages
! pip install patchify
! pip install tifffile

### Electron Microscopic Image Dataset

The data can be download from https://www.epfl.ch/labs/cvlab/data/data-em/

The whole data is a 3D tiff image with 1065x2048x1536 volume where each voxel is approximately 5x5x5nm

### Creating small patches 
First we split the images into smaller patches using patchify

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from Utils import Create_Small_Patches
import tifffile as tiff

In [4]:
training_image_stack = tiff.imread('training.tif')
training_mask_stack = tiff.imread('training_groundtruth.tif')
test_image_stack = tiff.imread('testing.tif')
test_mask_stack = tiff.imread('testing_groundtruth.tif')

In [5]:
Create_Small_Patches(training_image_stack,'Data/Training/Images/')
Create_Small_Patches(training_mask_stack,'Data/Training/Masks/')
Create_Small_Patches(test_image_stack,'Data/Test/Images/')
Create_Small_Patches(test_mask_stack,'Data/Test/Masks/')